In [ ]:
# the objective of this notebook is to collect data from 2 APIs and store them as csv Files 
# The first API will be the Universalis API to collect 
# Historical market data in the consumable market in FFXIV
# The second API will be the XIVAPI to collect
# Item data and World data for the consumable market in FFXIV
# The data will be stored in csv files for further analysis

# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Setting the working directory
os.chdir('/home/andres/Documents/FFXIV')

# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'

